In [1]:
import os 
print(os.getenv("CONDA_DEFAULT_ENV"))
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

stable_env


In [2]:
import torch
import json
import datasets
import datetime

In [3]:
from tqdm.auto import tqdm
from datasets import Dataset
from functools import partial
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader

In [4]:
from util.vision_util import process_vision_info
from util.logutil import init_logger, get_logger

### Load the MultiDomain Dataset

In [5]:
prefix = "Generate a one word or single number answer for the given image and question"

In [6]:
def prepend_prefix(example):
    example['question'] = prefix + ': ' + example['question']
    return example

In [7]:
dataset = load_dataset("dutta18/multi-domain-VQA-1.5K")

In [8]:
train_set, val_set = dataset['train'], dataset['validation']

In [9]:
train_set = train_set.map(prepend_prefix)
val_set = val_set.map(prepend_prefix)

In [10]:
len(train_set), len(val_set)

(1500, 600)

### Creating JSON Format of the AOKVQA

In [11]:
formattedJSONTrain = list()

for idx in tqdm(range(len(train_set))):
    currentJSON =   {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "image": train_set[idx]['image']
                        },
                        {"type": "text", "text": f"{train_set[idx]['question']}"}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": f"{train_set[idx]['answer']}"}
                    ]
                }
            ]
        }  
    formattedJSONTrain.append(currentJSON)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [12]:
formattedJSONVal = list()

for idx in tqdm(range(len(val_set))):
    currentJSON =   {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "image": val_set[idx]['image']
                        },
                        {"type": "text", "text": f"{val_set[idx]['question']}"}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": f"{val_set[idx]['answer']}"}
                    ]
                }
            ]
        }  
    formattedJSONVal.append(currentJSON)

  0%|          | 0/600 [00:00<?, ?it/s]

In [13]:
formattedJSONTrain[0], formattedJSONVal[0]

({'messages': [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=792x525>},
     {'type': 'text',
      'text': 'Generate a one word or single number answer for the given image and question: how many antitrypsin was talc used to sclerose emphysematous lung, alpha-deficiency?'}]},
   {'role': 'assistant', 'content': [{'type': 'text', 'text': '1'}]}]},
 {'messages': [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427>},
     {'type': 'text',
      'text': 'Generate a one word or single number answer for the given image and question: How many bowl are there?'}]},
   {'role': 'assistant', 'content': [{'type': 'text', 'text': 'one'}]}]})

In [14]:
output_dir = f'train_output/{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}/'
init_logger(output_dir)
logger = get_logger()

In [15]:
device = "cuda"

### Prepare Dataloaders

In [16]:
from torch.utils.data import Dataset

In [17]:
class aokvqa(Dataset):
    def __init__(self, formatted_json_data):
        super().__init__()
        self.data = formatted_json_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [18]:
train_dataset = aokvqa(formattedJSONTrain)
val_dataset = aokvqa(formattedJSONVal)

In [19]:
def find_assistant_content_sublist_indexes(l):
    '''
    A message from train_data/data.json may look like below:
        {
            "messages": [
                {'role': 'user', 'content': [{'type': 'image', 'image': 'train_data/1.jpeg'}, {'type': 'text', 'text': '描述一下这个图片'}]}, 
                {'role': 'assistant', 'content': [{'type': 'text', 'text': '这张图片展示了一位年轻女子和她的狗在海滩上玩耍的场景。女子穿着格子衬衫和黑色裤子，坐在沙滩上，与她的金毛犬互动。她们的手臂伸展着，似乎在进行某种游戏或训练。背景是广阔的海洋和晴朗的天空，阳光洒在沙滩上，营造出温暖而宁静的氛围。整体画面充满了快乐和放松的感觉。'}]}
            ]
        }
    After apply_chat_template, the text will look like below:
        ['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>描述一下这个图片<|im_end|>\n<|im_start|>assistant\n这张图片展示了一位年轻女子和她的狗在海滩上玩耍的场景。女子穿着格子衬衫和黑色裤子，坐在沙滩上，与她的金毛犬互动。她们的手臂伸展着，似乎在进行某种游戏或训练。背景是广阔的海洋和晴朗的天空，阳光洒在沙滩上，营造出温暖而宁静的氛围。整体画面充满了快乐和放松的感觉。<|im_end|>\n']

    This function tries to find the indexes of the assistant content in the input_ids list to build labels.
    '''
    start_indexes = []
    end_indexes = []

    # Iterate through the list to find starting points
    for i in range(len(l) - 1):
        # Check if the current and next elements form the start sequence
        if l[i] == 151644 and l[i+1] == 77091 and l[i+2] == 198:
            start_indexes.append(i+3)
            # Now look for the first 151645 and 198 after the start
            for j in range(i+3, len(l)-1):
                if l[j] == 151645 and l[j+1] == 198:
                    end_indexes.append(j+2) # **NOTE** the <|im_end|>\n 2 tokens should be included in the label, so that model can predicate end of output.
                    break  # Move to the next start after finding the end

    return list(zip(start_indexes, end_indexes))

In [20]:
def collate_fn(batch, processor, device):
    
    messages = [m['messages'] for m in batch]
    texts = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=False) for msg in messages]
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=texts,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    inputs = inputs.to(device)

    input_ids_lists = inputs['input_ids'].tolist()
    assert len(messages) == len(input_ids_lists)

    labels_list = []
    for ids_list in input_ids_lists:
        label_ids = [-100] * len(ids_list)
        for begin_end_indexs in find_assistant_content_sublist_indexes(ids_list):
            label_ids[begin_end_indexs[0]:begin_end_indexs[1]] = ids_list[begin_end_indexs[0]:begin_end_indexs[1]]
        labels_list.append(label_ids)

    labels_ids = torch.tensor(labels_list, dtype=torch.int64)
    return inputs, labels_ids

In [21]:
def write_chat_template(processor, output_dir):
    '''
    ***Note**

    We should have not had this function, as normal processor.save_pretrained(output_dir) would save chat_template.json file.
    However, on 2024/09/05, I think a commit introduced a bug to "huggingface/transformers", which caused the chat_template.json file not to be saved. 
    See the below commit, src/transformers/processing_utils.py line 393, this commit avoided chat_template.json to be saved.
    https://github.com/huggingface/transformers/commit/43df47d8e78238021a4273746fc469336f948314#diff-6505546ec5a9ab74b2ce6511681dd31194eb91e9fa3ce26282e487a5e61f9356

    To walk around that bug, we need manually save the chat_template.json file.

    I hope this bug will be fixed soon and I can remove this function then.
    '''
    output_chat_template_file = os.path.join(output_dir, "chat_template.json")
    chat_template_json_string = json.dumps({"chat_template": processor.chat_template}, indent=2, sort_keys=True) + "\n"
    with open(output_chat_template_file, "w", encoding="utf-8") as writer:
        writer.write(chat_template_json_string)
        logger.info(f"chat template saved in {output_chat_template_file}")

### Initialize Quantization Configs

In [22]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig

In [23]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory savings
    bnb_4bit_quant_type="nf4",  # Use NormalFloat4 (NF4) quantization type
)

### Model Loading

In [24]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.float16,
    low_cpu_mem_usage = True,
    attn_implementation="flash_attention_2",
    quantization_config=bnb_config,
    device_map='auto'
)
# Load processor. 
# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=256*28*28, max_pixels=512*28*28, padding_side="left", use_fast=True)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Initialize DORA Configs

In [25]:
dora_config = LoraConfig(
    r=8,
    lora_alpha=8*2,  # Scaling factor
    lora_dropout=0.05,  # Dropout rate
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "qkv", "attn.proj"], 
    task_type="CAUSAL_LM",
    use_dora = True
)

#### Apply Grad Checkpointing using and optimizations: prepare_model_for_kbit_training()

In [26]:
model = prepare_model_for_kbit_training(model)
qdora_qwen_model = get_peft_model(model, dora_config)

In [27]:
#qdora_qwen_model

### Report Parameter Size: ~ 12.0 M

In [28]:
def report_trainable_params():
    
    trainable = sum(p.numel() for p in qdora_qwen_model.parameters() if p.requires_grad)
    print(f"Total trainable params: {trainable/1e6:.1f} M")

In [29]:
report_trainable_params()

Total trainable params: 12.0 M


### Create & Test Dataloader

In [30]:
batchSize_ = 4

In [31]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batchSize_,
    collate_fn=partial(collate_fn, processor=processor, device=device),
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batchSize_,
    collate_fn=partial(collate_fn, processor=processor, device=device)
)

### Validation Function

In [32]:
def do_validation():
    
    qdora_qwen_model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(val_loader):
            inputs, labels = batch
            outputs = qdora_qwen_model(**inputs, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    qdora_qwen_model.train()
    torch.cuda.empty_cache()
    return avg_val_loss

### Training Hyperparams

In [33]:
from transformers import get_cosine_schedule_with_warmup

In [34]:
LR = 1e-4
epochs = 5
weight_decay = 0.01
gradient_accumulation_steps = 2

In [35]:
global_step = 0
best_val_loss = float("inf")

In [36]:
steps_per_epoch     = len(train_loader) // gradient_accumulation_steps
total_train_steps   = steps_per_epoch * epochs
num_warmup_steps    = int(0.05 * total_train_steps)          # 5 %   (quick)  
# ➟ for medium: 0.03 works fine

In [37]:
print(total_train_steps, num_warmup_steps)

935 46


In [38]:
optimizer = AdamW(qdora_qwen_model.parameters(), lr=LR, weight_decay=weight_decay)

In [39]:
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=total_train_steps,
)

In [40]:
saveDir = '/home/aritrad/main/Qwen2-VL-2B/MOE/Multidomain/chkpts'

In [41]:
qdora_qwen_model.use_cache = False

## Native PyTorch Training Loop

##### I am using val_loss as the checkpointing criteria, but any other metric which test text generation quality can be used here.
##### MAX GPU USAGE = 22 GB on NVIDIA A40 Card (Adjust LORA/DORA Rank, batch size and accumulation steps accordingly)

In [ ]:
for epoch in tqdm(range(epochs)):

    accumulated_loss = 0
    
    for idx, batch in enumerate(train_loader):
        
        inputs, labels = batch
        outputs = qdora_qwen_model(**inputs, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
    
        loss.backward()
        accumulated_loss += loss.item()
        
        if (idx+1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1

            logger.info(f"[ Epoch {epoch+1} | idx: {idx} | Optim Step {global_step} | Train Loss: {loss.item():.4f} ]")

            if global_step % 60 == 0:
                avg_val_loss = do_validation()
                logger.info(f"Val Loss @ Optim step: {global_step} -> {avg_val_loss:.4f}\n")
            
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    qdora_qwen_model.save_pretrained(os.path.join(saveDir, 'Qwen-MultiDomain-QDORA-chkpt-16R.pt'))
                    logger.info(f"***** ✅ Checkpoint Saved *****\n")

    logger.info(f"Epoch {epoch+1} completed. Avg loss: {accumulated_loss / len(train_loader):.4f}")

  0%|          | 0/5 [00:00<?, ?it/s]

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
2025-07-08 05:43:31-3080550328.py:20-INFO >> [ Epoch 1 | idx: 1 | Optim Step 1 | Train Loss: 0.6400 ]
2025-07-08 05:43:38-3080550328.py:20-INFO >> [ Epoch 1 | idx: 3 | Optim Step 2 | Train Loss: 0.4567 ]
2025-07-08 05:43:46-3080550328.py:20-INFO >> [ Epoch 1 | idx: 5 | Optim Step 3 | Train Loss: 1.1952 ]
2025-07-08 05:43:53-3080550328.py:20-INFO >> [ Epoch 1 | idx: 7 | Optim Step 4 | Train Loss: 1.3454 ]
2025-07-08 05:44:01-3080550328.py:20-INFO >> [ Epoch 1 | idx: 9 | Optim Step 5 | Train Loss: 0.7239 ]
2025-07-08 05:44:08-3080550328.py:20-INFO >> [ Epoch 1 | idx: 11 | Optim Step 6 | Train Loss: 1.2187 ]
2025-07-08 05:44:16-3080550328.py:20-INFO >> [ Epoch 1 | idx: 13 | Optim Step 7 | Train Loss: 2.7951 ]
2025-07-08 05:44:23-3080550328.py:20-INFO >> [ Epoch 1 | idx: 15 | Optim Step 8

  0%|          | 0/150 [00:00<?, ?it/s]

2025-07-08 05:54:28-3080550328.py:24-INFO >> Val Loss @ Optim step: 60 -> 1.4042

2025-07-08 05:54:30-3080550328.py:29-INFO >> ***** ✅ Checkpoint Saved *****

/home/aritrad/miniconda3/envs/stable_env/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/aritrad/miniconda3/envs/stable_env/lib/python3.9/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
2025-07-08 05:54:37-3080550328.py:20-INFO >> [ Epoch

  0%|          | 0/150 [00:00<?, ?it/s]